In [4]:
  %pip install PyQt5

   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ------------ --------------------------- 2.1/6.9 MB 11.7 MB/s eta 0:00:01
   ------------------------- -------------- 4.5/6.9 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------  6.8/6.9 MB 12.0 MB/s eta 0:00:01
   ---------------------------------------- 6.9/6.9 MB 11.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.1 MB ? eta -:--:--
   - -------------------------------------- 2.4/50.1 MB 12.2 MB/s eta 0:00:04
   --- ------------------------------------ 4.7/50.1 MB 11.9 MB/s eta 0:00:04
   ----- ---------------------------------- 7.3/50.1 MB 11.9 MB/s eta 0:00:04
   ------- -------------------------------- 9.7/50.1 MB 11.8 MB/s eta 0:00:04
   --------- ------------------------------ 12.3/50.1 MB 11.9 MB/s eta 0:00:04
   ----------- ---------------------------- 14.7/50.1 MB 11.8 MB/s eta 0:00:03
   ------------- -------------------------- 17.3/50.1 MB 11.8 MB/s eta 0:00:03
   -

DEPRECATION: Loading egg at c:\users\administrator\appdata\local\programs\python\python312\lib\site-packages\torchlight-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


## COCO → Yolo 데이터 구조 변환

In [ ]:
import json
import os
from PIL import Image

def convert_bbox(img_width, img_height, bbox):
    x = bbox['x']
    y = bbox['y']
    width = bbox['width']
    height = bbox['height']
    
    x_center = (x + width / 2) / img_width
    y_center = (y + height / 2) / img_height
    width = width / img_width
    height = height / img_height
    
    return x_center, y_center, width, height

def convert_keypoint(img_width, img_height, keypoint):
    x = float(keypoint['x']) / img_width
    y = float(keypoint['y']) / img_height
    return x, y

def convert_json_to_yolo(json_file, img_folder, output_dir):
    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except UnicodeDecodeError:
        # UTF-8로 읽기 실패 시 다른 인코딩 시도
        with open(json_file, 'r', encoding='cp949') as f:
            data = json.load(f)
    
    os.makedirs(output_dir, exist_ok=True)
    
    missing_images = []
    
    for annotation in data['annotations']:
        frame_number = annotation['frame_number']
        timestamp = annotation['timestamp']
        
        img_filename = f"frame_{frame_number}_timestamp_{timestamp}.jpg"
        img_path = os.path.join(img_folder, img_filename)
        
        if not os.path.exists(img_path):
            missing_images.append(img_filename)
            continue
        
        try:
            with Image.open(img_path) as img:
                img_width, img_height = img.size
        except Exception as e:
            print(f"이미지 파일을 열 수 없습니다: {img_path}. 오류: {e}")
            continue
        
        txt_filename = f"frame_{frame_number}_timestamp_{timestamp}.txt"
        txt_path = os.path.join(output_dir, txt_filename)
        
        with open(txt_path, 'w') as f:
            class_id = 0
            
            bbox = annotation['bounding_box']
            x_center, y_center, width, height = convert_bbox(img_width, img_height, bbox)
            
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
            
            keypoints = annotation['keypoints']
            for i in range(1, 16):
                kp = keypoints.get(str(i))
                if kp:
                    x, y = convert_keypoint(img_width, img_height, kp)
                    f.write(f" {x:.6f} {y:.6f}")
                else:
                    f.write(" 0 0")
            
            f.write("\n")
        
        print(f"변환 완료: {txt_path}")
    
    if missing_images:
        print("\n누락된 이미지 파일:")
        for img in missing_images:
            print(img)
        print(f"\n총 {len(missing_images)}개의 이미지 파일이 누락되었습니다.")

# 사용 예
json_file = r"D:\hackerton_6th\dataset\test\cat-arch-010988.json"
img_folder = r"D:\hackerton_6th\dataset\test\cat-arch-010988"
output_dir = r"D:\hackerton_6th\dataset\test\yolo_labels"

convert_json_to_yolo(json_file, img_folder, output_dir)

In [ ]:
# yaml 파일생성 추가
import yaml
import shutil

def create_yaml_file(output_base_dir, class_mapping):
    yaml_content = {
        'path': output_base_dir,  # 데이터셋의 루트 디렉토리
        'train': 'images/train',  # 학습 이미지 경로 (상대 경로)
        'val': 'images/val',      # 검증 이미지 경로 (상대 경로)
        'test': 'images/test',    # 테스트 이미지 경로 (선택사항)
        'nc': len(class_mapping), # 클래스 수
        'names': list(class_mapping.keys())  # 클래스 이름 리스트
    }

    yaml_path = os.path.join(output_base_dir, 'dataset.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False)

    print(f"YAML 파일이 생성되었습니다: {yaml_path}")

def process_all_json_files(base_dir, output_base_dir):
    class_mapping = {}
    class_counter = 0
    all_images_dir = os.path.join(output_base_dir, 'images')
    all_labels_dir = os.path.join(output_base_dir, 'labels')
    os.makedirs(all_images_dir, exist_ok=True)
    os.makedirs(all_labels_dir, exist_ok=True)

    for class_name in os.listdir(base_dir):
        class_dir = os.path.join(base_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        if class_name not in class_mapping:
            class_mapping[class_name] = class_counter
            class_counter += 1

        class_id = class_mapping[class_name]

        for file in os.listdir(class_dir):
            if file.endswith('.json'):
                json_file = os.path.join(class_dir, file)
                img_folder = os.path.join(class_dir, file[:-5])  # Remove .json extension

                if not os.path.exists(img_folder):
                    print(f"이미지 폴더를 찾을 수 없습니다: {img_folder}")
                    continue

                # YOLO 형식으로 변환
                convert_json_to_yolo(json_file, img_folder, all_labels_dir, class_id)

                # 이미지 복사
                for img in os.listdir(img_folder):
                    if img.endswith(('.jpg', '.jpeg', '.png')):
                        shutil.copy(os.path.join(img_folder, img), all_images_dir)

    # 클래스 매핑 정보 저장
    with open(os.path.join(output_base_dir, 'class_mapping.txt'), 'w') as f:
        for class_name, class_id in class_mapping.items():
            f.write(f"{class_id}: {class_name}\n")

    # YAML 파일 생성
    create_yaml_file(output_base_dir, class_mapping)

# 사용 예
base_dir = r"D:\hackerton_6th\Pretrained dataset\coco\Training\cat"
output_base_dir = r"D:\hackerton_6th\dataset\YOLO_output"

process_all_json_files(base_dir, output_base_dir)

In [ ]:
# 메타데이터 포함
import json
import os
import shutil
import yaml

def convert_bbox(img_width, img_height, bbox):
    x = bbox['x'] / img_width
    y = bbox['y'] / img_height
    w = bbox['width'] / img_width
    h = bbox['height'] / img_height
    return x + w/2, y + h/2, w, h

def convert_keypoint(img_width, img_height, keypoint):
    x = float(keypoint['x']) / img_width
    y = float(keypoint['y']) / img_height
    return x, y

def convert_json_to_yolo(json_file, img_folder, output_dir, class_id):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    os.makedirs(output_dir, exist_ok=True)
    
    metadata = data['metadata']
    metadata_output = os.path.join(output_dir, 'metadata')
    os.makedirs(metadata_output, exist_ok=True)

    for annotation in data['annotations']:
        frame_number = annotation['frame_number']
        timestamp = annotation['timestamp']
        
        img_filename = f"frame_{frame_number}_timestamp_{timestamp}.jpg"
        img_path = os.path.join(img_folder, img_filename)
        
        if not os.path.exists(img_path):
            print(f"이미지 파일을 찾을 수 없습니다: {img_path}")
            continue
        
        with Image.open(img_path) as img:
            img_width, img_height = img.size
        
        txt_filename = img_filename.replace('.jpg', '.txt')
        txt_path = os.path.join(output_dir, txt_filename)
        
        with open(txt_path, 'w') as f:
            bbox = annotation['bounding_box']
            x_center, y_center, width, height = convert_bbox(img_width, img_height, bbox)
            
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
            
            keypoints = annotation['keypoints']
            for i in range(1, 16):
                kp = keypoints.get(str(i))
                if kp:
                    x, y = convert_keypoint(img_width, img_height, kp)
                    f.write(f" {x:.6f} {y:.6f}")
                else:
                    f.write(" 0 0")
            
            f.write("\n")
        
        # 메타데이터 저장
        metadata_filename = img_filename.replace('.jpg', '_metadata.json')
        metadata_path = os.path.join(metadata_output, metadata_filename)
        frame_metadata = {
            "frame_info": {
                "frame_number": frame_number,
                "timestamp": timestamp
            },
            "metadata": metadata
        }
        with open(metadata_path, 'w', encoding='utf-8') as f:
            json.dump(frame_metadata, f, ensure_ascii=False, indent=2)
        
        # 이미지 복사
        shutil.copy(img_path, os.path.join(output_dir, img_filename))
        
    print(f"변환 완료: {json_file}")

def process_all_json_files(base_dir, output_base_dir):
    class_mapping = {}
    class_counter = 0

    for class_name in os.listdir(base_dir):
        class_dir = os.path.join(base_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        if class_name not in class_mapping:
            class_mapping[class_name] = class_counter
            class_counter += 1

        class_id = class_mapping[class_name]
        class_output_dir = os.path.join(output_base_dir, class_name)

        for file in os.listdir(class_dir):
            if file.endswith('.json'):
                json_file = os.path.join(class_dir, file)
                img_folder = os.path.join(class_dir, file[:-5])  # Remove .json extension

                if not os.path.exists(img_folder):
                    print(f"이미지 폴더를 찾을 수 없습니다: {img_folder}")
                    continue

                convert_json_to_yolo(json_file, img_folder, class_output_dir, class_id)

    # YAML 파일 생성
    yaml_content = {
        'path': output_base_dir,
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'nc': len(class_mapping),
        'names': list(class_mapping.keys()),
        'metadata': 'metadata'  # 메타데이터 폴더 경로 추가
    }

    yaml_path = os.path.join(output_base_dir, 'dataset.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False)

    print(f"YAML 파일이 생성되었습니다: {yaml_path}")

# 사용 예
base_dir = r"D:\hackerton_6th\Pretrained dataset\coco\Training\cat"
output_base_dir = r"D:\hackerton_6th\dataset\YOLO_output"

process_all_json_files(base_dir, output_base_dir)

In [ ]:
# LLM 입력 준비 (메타데이터 활용 예시)
def prepare_llm_input(image_path, label_path, metadata_path):
    # YOLO 라벨 읽기
    with open(label_path, 'r') as f:
        yolo_data = f.read().strip().split()
    
    # 메타데이터 읽기
    with open(metadata_path, 'r', encoding='utf-8') as f:
        metadata = json.load(f)
    
    # LLM 입력 준비
    prompt = f"""
    고양이 이미지 분석:
    
    YOLO 감지 결과:
    클래스 ID: {yolo_data[0]}
    바운딩 박스: {' '.join(yolo_data[1:5])}
    키포인트: {' '.join(yolo_data[5:])}
    
    메타데이터:
    행동: {metadata['metadata']['action']}
    감정: {metadata['metadata']['inspect']['emotion']}
    통증/질병 여부: {metadata['metadata']['inspect']['painDisease']}
    비정상 행동 여부: {metadata['metadata']['inspect']['abnormalAction']}
    
    위 정보를 바탕으로 고양이의 상태를 분석하고, 이러한 포즈가 의미하는 바를 설명해주세요.
    """
    
    return prompt

# 사용 예
image_path = 'path/to/image.jpg'
label_path = 'path/to/label.txt'
metadata_path = 'path/to/metadata.json'

llm_input = prepare_llm_input(image_path, label_path, metadata_path)
# 이제 llm_input을 LLM에 전달하여 분석 결과를 얻을 수 있습니다.

#### 이미지 + BBox, KeyPoint 매핑 TEST

In [1]:
import cv2
import numpy as np
import os

def draw_label_on_image(img_path, label_path, output_path):
    # 이미지 읽기
    img = cv2.imread(img_path)
    height, width = img.shape[:2]

    # 라벨 파일 읽기
    with open(label_path, 'r') as f:
        label_data = f.read().strip().split()

    # 클래스 ID (여기서는 사용하지 않음)
    class_id = int(label_data[0])

    # 바운딩 박스 정보
    x_center, y_center, box_width, box_height = map(float, label_data[1:5])
    x1 = int((x_center - box_width/2) * width)
    y1 = int((y_center - box_height/2) * height)
    x2 = int((x_center + box_width/2) * width)
    y2 = int((y_center + box_height/2) * height)

    # 바운딩 박스 그리기
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # 키포인트 그리기
    keypoints = list(map(float, label_data[5:]))
    for i in range(0, len(keypoints), 2):
        x = int(keypoints[i] * width)
        y = int(keypoints[i+1] * height)
        cv2.circle(img, (x, y), 3, (0, 0, 255), -1)

    # 결과 이미지 저장
    cv2.imwrite(output_path, img)
    print(f"이미지가 저장되었습니다: {output_path}")

# 사용 예
img_folder = r"D:\hackerton_6th\dataset\test\cat-arch-010988"
label_folder = r"D:\hackerton_6th\dataset\test\yolo_labels"
output_folder = r"D:\hackerton_6th\dataset\test\visualized_labels"

os.makedirs(output_folder, exist_ok=True)

for label_file in os.listdir(label_folder):
    if label_file.endswith('.txt'):
        img_file = label_file.replace('.txt', '.jpg')
        img_path = os.path.join(img_folder, img_file)
        label_path = os.path.join(label_folder, label_file)
        output_path = os.path.join(output_folder, f"visualized_{img_file}")

        if os.path.exists(img_path):
            draw_label_on_image(img_path, label_path, output_path)
        else:
            print(f"이미지 파일을 찾을 수 없습니다: {img_path}")

print("모든 이미지 처리가 완료되었습니다.")

이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_0_timestamp_0.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_100_timestamp_4000.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_105_timestamp_4200.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_10_timestamp_400.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_110_timestamp_4400.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_115_timestamp_4600.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_120_timestamp_4800.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_125_timestamp_5000.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_130_timestamp_5200.jpg
이미지가 저장되었습니다: D:\hackerton_6th\dataset\test\visualized_labels\visualized_frame_135_timest